In [17]:
import pandas as pd
import numpy as np
import csv
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
import nltk
import matplotlib.pyplot as plt
from scipy import sparse
# Code source: https://degravek.github.io/project-pages/project1/2017/04/28/New-Notebook/
# Dataset from Chakraborty et al. (https://github.com/bhargaviparanjape/clickbait/tree/master/dataset)

In [28]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import  LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import *
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

class SentimentAnalysis():

    def __init__(self):        

        columnNames = ["jsonid", "label", "headline_text", "subject", "speaker", "speaker_job_title", "state_info", "party_affiliation", "barely_true_counts", "false_counts", "half_true_counts", "mostly_true_counts", "pants_on_fire_counts", "context","clean", "sentiment_vector","vader_polarity", "sentiment_score"]
        dataTrain = pd.read_csv('input_data/train_sentiment.csv', sep=',', header=None, names = columnNames)
        dataTest = pd.read_csv('input_data/test_sentiment.csv', sep=',', header=None, names = columnNames)

        #dropping columns
        columnsToRemove = ['jsonid', 'label', 'subject', 'speaker','speaker_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context', 'sentiment_vector']
        dataTrain = dataTrain.drop(columns=columnsToRemove)
        dataTest = dataTest.drop(columns=columnsToRemove)
        dataTrain = dataTrain.iloc[1:] 
        dataTest = dataTest.iloc[1:]
    
    
        tfidfV = TfidfVectorizer(stop_words='english', min_df=5, max_df=30, use_idf=True, smooth_idf=True, token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')

        '''
        self.logR_pipeline = Pipeline([
                ('LogRCV', tfidfV),
                ('LogR_clf',LogisticRegression(solver='liblinear', C=32/100))
                ])
        '''
        
        self.logR_pipeline = Pipeline([
                ('LogRCV', tfidfV),
                ('LogR_clf',RandomForestClassifier(n_estimators=100, max_depth=8, random_state=0))
                ])
        

        self.logR_pipeline.fit(dataTrain['headline_text'],dataTrain['vader_polarity'])
        predicted_LogR = self.logR_pipeline.predict(dataTest['headline_text'])
        score = metrics.accuracy_score(dataTest['vader_polarity'], predicted_LogR)
        print("Sentiment Analysis Model Trained - accuracy:   %0.6f" % score)
        

    def predict(self, text):
        predicted = self.logR_pipeline.predict([text])
        predicedProb = self.logR_pipeline.predict_proba([text])[:,1]
        return bool(predicted), float(predicedProb)
    
    
sa = SentimentAnalysis()
sa.predict("Says the Annies List political group supports third-trimester abortions on demand.")

Sentiment Analysis Model Trained - accuracy:   0.960784


(True, 0.9811315920552556)

In [34]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import  LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import *
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

class SentimentAnalysis():

    def __init__(self):        

        columnNames = ["jsonid", "label", "headline_text", "subject", "speaker", "speaker_job_title", "state_info", "party_affiliation", "barely_true_counts", "false_counts", "half_true_counts", "mostly_true_counts", "pants_on_fire_counts", "context","clean", "sentiment_vector","vader_polarity", "sentiment_score"]
        sentimentTrain = pd.read_csv('input_data/sentiment/my_train_with_raw.csv')
        sentimentTest = pd.read_csv('input_data/sentiment/my_test_with_raw.csv')
        
        train = pd.read_csv('input_data/train_sentiment.csv')
        test = pd.read_csv('input_data/test_sentiment.csv')
        
        train = train.merge(sentimentTrain, how = 'outer')
        test = test.merge(sentimentTest, how = 'outer')
        
        train.head()
        test.head()
        
        #dropping columns
        #  = ['jsonid', 'label', 'subject', 'speaker','speaker_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context', 'sentiment_vector']
        #dataTrain = dataTrain.drop(columns=columnsToRemove)
        #dataTest = dataTest.drop(columns=columnsToRemove)
        
        tfidfV = TfidfVectorizer(stop_words='english', min_df=5, max_df=30, use_idf=True, smooth_idf=True, token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')

        self.logR_pipeline = Pipeline([
                ('LogRCV', tfidfV),
                ('LogR_clf',LogisticRegression(solver='liblinear', C=32/100))
                ])
        
        self.logR_pipeline = Pipeline([
                ('LogRCV', tfidfV),
                ('LogR_clf',RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0))
                ])

        self.logR_pipeline.fit(train['headline_text'],train['polarity'])
        predicted_LogR = self.logR_pipeline.predict(test['headline_text'])
        score = metrics.accuracy_score(test['polarity'], predicted_LogR)
        print("Sentiment Analysis Model Trained - accuracy:   %0.6f" % score)
        

    def predict(self, text):
        predicted = self.logR_pipeline.predict([text])
        predicedProb = self.logR_pipeline.predict_proba([text])[:,1]
        return bool(predicted), float(predicedProb)
    
    
sa = SentimentAnalysis()
sa.predict("Says the Annies List political group supports third-trimester abortions on demand.")

Sentiment Analysis Model Trained - accuracy:   0.618309


(False, 0.6379006769704134)

In [11]:
sentimentTrain = pd.read_csv('input_data/sentiment_train.csv')
sentimentTest = pd.read_csv('input_data/sentiment_test.csv')

train = pd.read_csv('input_data/train_sentiment.csv')
test = pd.read_csv('input_data/test_sentiment.csv')

train_merge = train.merge(sentimentTrain, on = 'clean', how = 'outer')
test_merge = test.merge(sentimentTest, on = 'clean', how = 'outer')

print(test_merge.shape)
test_merge.head()

(1289, 22)


,Unnamed: 0_x,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,...,mostlytrueocunts,pantsonfirecounts,context,clean,sentiment_vector,vader_polarity,sentiment_score,Unnamed: 0_y,sentiment,polarity
0,80.0,2854.json,pants-fire,Rebuilding three high schools will benefit 40 ...,education,carole-smith,Portland schools superintendent,Oregon,none,0.0,...,0.0,1.0,a press briefing,rebuild three high school benefit percent port...,"[0.0, 0.75, 0.25, 0.4588]",0.0,0.4588,80,0.1,0
1,124.0,10147.json,false,On an income cap for recipients of the popular...,education,jason-carter,State Senator,Georgia,democrat,1.0,...,1.0,0.0,forum,incom cap recipe popular hope scholarship,"[0.0, 0.412, 0.588, 0.6908]",1.0,0.6908,124,0.2,0
2,168.0,7744.json,mostly-true,Says states mandated tests come from an Englis...,education,george-lavender,State representative,Texas,republican,0.0,...,1.0,0.0,in Texas House of Representatives debate,say state mandat test come english company,"[0.0, 1.0, 0.0, 0.0]",0.0,0.0000,168,0.0,0
3,181.0,5470.json,true,Pasco County schools have graduation rates sub...,education,heather-fiorentino,superintendent of Pasco County's public schools,Florida,republican,0.0,...,0.0,1.0,a press release,pisco county school graduate rate substantia h...,"[0.0, 1.0, 0.0, 0.0]",0.0,0.0000,181,-0.3,-1
4,187.0,8320.json,mostly-true,Today many Florida teachers are at risk of hav...,education,charlie-crist,0,Florida,democrat,15.0,...,19.0,2.0,a Tampa Bay Times op-ed,today mani florida teacher risk pay impact per...,"[0.241, 0.759, 0.0, -0.3612]",0.0,-0.3612,187,-0.3,-1


In [6]:
sentimentTrain.head()

,Unnamed: 0,clean,sentiment,polarity
0,0,say anni list polit group support third trimme...,-0.7,-1
1,1,decline coal start start natur gas took start ...,-0.4,-1
2,2,hillary clinton agre john mccain vote give geo...,0.2,0
3,3,health care reform legis like mandat free sex ...,0.2,0
4,4,econom turnaround start end term,0.1,0


In [32]:
temp =  pd.read_csv('input_data/sentiment/my_train_with_raw.csv')
train = pd.read_csv('input_data/train_sentiment.csv')
temp = train.merge(temp, how = 'outer')

temp.shape

(10263, 22)

In [39]:
books =pd.read_csv('input_data/sentiment/books/negative.review')
books.head()

,avid:1 your:1 horrible_book:1 wasted:1 use_it:1 the_entire:1 money.i:1 i_lit:1 i_read:1 lit:1 i_would:1 relationship:1 read:1 a_<num>:1 reader_and:1 reader:1 suffering:1 fire_one:1 i_had:1 year_old:2 gotten:1 horrible:3 lit_this:1 world...don't:1 my:2 one_star:1 headache_the:1 this_book:5 mom:1 was_horrible:1 friend:1 book_horrible:1 star_i:1 back:1 avid_reader:1 than_one:1 life:1 copy:1 rate_it:1 rate:1 my_mom:1 man:1 book_was:1 half:1 on_fire:1 and_then:1 reading_this:1 so:1 lower:1 i_could:1 <num>_year:2 than:1 time:2 half_of:1 time_spent:1 then:1 book:6 and_picked:1 possible:1 spent:1 old_man:1 up_after:1 one:2 horrible_if:1 one_less:1 part:1 was:2 entire:1 less_copy:1 to_rate:1 my_life:1 about_the:1 your_money.i:1 an_avid:1 if:1 the_relationship:1 use:1 a_headache:1 fire:1 lower_than:1 reading:1 a_friend:1 picked:1 purposes:1 then_got:1 waste_your:1 after_my:1 friend_i:1 old:2 man_and:1 and_i:1 world...don't_waste:1 book_on:1 part_about:1 copy_in:1 book_back:1 book_wasted:1 have_i:1 time_and:1 the_world...don't:1 better:1 if_it:1 star:1 got:1 mom_had:1 read_half:1 waste:1 after:1 i:6 about:1 could_use:1 had_gotten:1 was_possible:1 year:2 it_lower:1 relationship_the:1 wasted_my:1 wish:1 wish_i:1 boy:1 purposes_this:1 got_to:1 the_time:1 it_was:1 back_so:1 suffering_from:1 spent_reading:1 book_up:1 less:1 better_purposes:1 headache:1 possible_to:1 money.i_wish:1 for_better:1 it_suffering:1 the_part:1 gotten_it:1 picked_this:1 entire_time:1 old_boy:1 i_am:1 the_<num>:1 boy_had:1 <num>:2 so_i:1 #label#:negative
0,to_use:1 shallow:1 found:1 he_castigates:1 cas...
1,avid:1 your:1 horrible_book:1 wasted:1 use_it:...
2,book_seriously:1 we:1 days_couldn't:1 me_tell:...
3,"mass:1 only:1 he:2 help:1 ""jurisfiction"":1 lik..."
4,save_your:1 class_and:1 his_facts:1 opinions:1...


In [50]:
import glob

data = []

for files in glob.glob('/Users/jonathan/Downloads/aclImdb/train/neg/*.txt'):
    data.append(open(files, "r").read())

print(len(data))
neg_df = pd.DataFrame(data)

neg_df['sentiment'] = -1

neg_df.head()

data = []

for files in glob.glob('/Users/jonathan/Downloads/aclImdb/train/pos/*.txt'):
    data.append(open(files, "r").read())

print(len(data))
pos_df = pd.DataFrame(data)
pos_df['sentiment'] = 1
pos_df.head()


12500
12500


,0,sentiment
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


In [54]:
pos_df.columns = ['raw_text', 'sentiment']
neg_df.columns = ['raw_text', 'sentiment']
df = neg_df.merge(pos_df, how = 'outer')
print(df.shape)
df.head()

df.to_csv('input_data/imdb_sentiment.csv')

(25000, 2)


In [56]:
print(df.shape)
df.head()

(25000, 2)


,raw_text,sentiment
0,Working with one of the best Shakespeare sourc...,-1
1,"Well...tremors I, the original started off in ...",-1
2,Ouch! This one was a bit painful to sit throug...,-1
3,"I've seen some crappy movies in my life, but t...",-1
4,"""Carriers"" follows the exploits of two guys an...",-1


In [57]:
import re

def cleaning(raw_news):
    import nltk
    
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub("[^a-zA-Z]", " ", raw_news)
    
    # 2. Convert to lower case.
    news =  news.lower()
    
    # 3. Tokenize.
    news_words = nltk.word_tokenize( news)
    
    # 4. Convert the stopwords list to "set" data type.
    stops = set(nltk.corpus.stopwords.words("english"))
    
    # 5. Remove stop words. 
    words = [w for w in  news_words  if not w in stops]
    
    # 6. Lemmentize 
    wordnet_lem = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    
    # 7. Stemming
    stems = [nltk.stem.SnowballStemmer('english').stem(w) for w in wordnet_lem ]
    
    # 8. Join the stemmed words back into one string separated by space, and return the result.
    return " ".join(stems)